In [15]:
file_path = './data/test_log1.out'

with open(file_path, 'r') as file:
    file_contents = file.read()
    file_lines = '\n'.join(file_contents.splitlines()[2000:3000])

In [16]:
file_lines

'Nov 09 13:11:41 CMX50070-101776 health_service[555]: removing /var/log/coredump/core.CMXmarsServer.1000.b98ef10ab3f8471aa548add9b9e5d223.86893.1699426915000000.lz4\nNov 09 13:11:41 CMX50070-101776 health_service[555]: removing /var/log/coredump/core.CMXmarsServer.1000.b98ef10ab3f8471aa548add9b9e5d223.84495.1699426893000000.lz4\nNov 09 13:11:41 CMX50070-101776 health_service[555]: removing /var/log/coredump/core.CMXmarsServer.1000.b98ef10ab3f8471aa548add9b9e5d223.69194.1699426870000000.lz4\nNov 09 13:11:41 CMX50070-101776 health_service[555]: removing /var/log/coredump/core.CMXmarsServer.1000.b98ef10ab3f8471aa548add9b9e5d223.3524180.1699426785000000.lz4\nNov 09 13:11:41 CMX50070-101776 health_service[555]: removing /var/log/coredump/core.CMXmarsServer.1000.b98ef10ab3f8471aa548add9b9e5d223.3522024.1699419845000000.lz4\nNov 09 13:11:41 CMX50070-101776 killall[1746]: chromium-bin: no process found\nNov 09 13:11:41 CMX50070-101776 systemd[1]: xserver-nodm.service: Control process exited, c

In [17]:
import nltk
from sentence_transformers import SentenceTransformer, util
import numpy as np
import torch

In [25]:
from sentence_transformers import SentenceTransformer
from sklearn.cluster import KMeans

embedder = SentenceTransformer('all-MiniLM-L6-v2')
embedder.max_seq_length = 512
corpus = file_lines.splitlines()
corpus_embeddings = embedder.encode(corpus)

# Perform kmean clustering
num_clusters = 20
clustering_model = KMeans(n_clusters=num_clusters)
clustering_model.fit(corpus_embeddings)
cluster_assignment = clustering_model.labels_

clustered_sentences = [[] for i in range(num_clusters)]
for sentence_id, cluster_id in enumerate(cluster_assignment):
    clustered_sentences[cluster_id].append(corpus[sentence_id])

for i, cluster in enumerate(clustered_sentences):
    print("Cluster ", i+1)
    print(cluster)
    print("")

/Users/hammerer/git/HackaTUM23/.rhode/lib/python3.11/site-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


Cluster  1
['Nov 09 13:11:42 CMX50070-101776 kernel: igb 0000:0d:00.0 eth_external: igb: eth_external NIC Link is Up 1000 Mbps Full Duplex, Flow Control: RX/TX', 'Nov 09 13:11:45 CMX50070-101776 kernel: CMX_COMMON_IOCTL_SET_GIGABIT_CHANNEL_LINK_CONFIG call', 'Nov 09 13:11:45 CMX50070-101776 kernel: --- CONFIG GIGABIT CHANNEL ---', 'Nov 09 13:11:45 CMX50070-101776 kernel: config_generic_gigabit_channel_link(): module_offset: 0x00040000 / sub_offset: 0x00020420 / channel_offset: 0x00000000  / confLengthDataPerAddr: 002', 'Nov 09 13:11:45 CMX50070-101776 kernel: config_generic_gigabit_channel_link(): access for address[000/001] = 0x00000014', "Nov 09 13:11:45 CMX50070-101776 kernel: config_generic_gigabit_channel_link(): access is 'clear_update_write'", 'Nov 09 13:11:45 CMX50070-101776 kernel: config_generic_gigabit_channel_link(): clear on 0x00060434 with value 0x00000000', 'Nov 09 13:11:45 CMX50070-101776 kernel: config_generic_gigabit_channel_link(): [000/001][000/002] 0x00000014 = 0x0

In [26]:
from scipy.spatial import distance
y_kmeans = clustering_model.predict(corpus_embeddings)

my_list=[]
for i in range(num_clusters):
    my_dict={}
    
    for j in range(len(y_kmeans)):
        
        if y_kmeans[j]==i:
            my_dict[j] =  distance.euclidean(clustering_model.cluster_centers_[i],corpus_embeddings[j])
    min_distance = min(my_dict.values())
    my_list.append(min(my_dict, key=my_dict.get))
 
summary = ""
for i in sorted(my_list):
    summary += corpus[i] + "\n"
print(summary)

Nov 09 13:11:42 CMX50070-101776 node[405]: Get facettes for mrt.base-helpk70...
Nov 09 13:11:42 CMX50070-101776 node[405]: Create TOC for mrt.base-helpk70...
Nov 09 13:11:42 CMX50070-101776 systemd-nspawn[891]: 
Nov 09 13:11:42 CMX50070-101776 Xserver[1884]:         to make sure that you have the latest version.
Nov 09 13:11:43 CMX50070-101776 node[405]: Get video meta info for CMX500_Using_data_services_HowTo_en_02.mp4...
Nov 09 13:11:45 CMX50070-101776 dockerd[1853]: time="2023-11-09T13:11:45.067827022Z" level=info msg="Loading containers: start."
Nov 09 13:11:45 CMX50070-101776 audit[2720]: NETFILTER_CFG table=filter family=2 entries=4 op=xt_replace pid=2720 comm="iptables"
Nov 09 13:11:45 CMX50070-101776 audit: PROCTITLE proctitle=2F7573722F7362696E2F69707461626C6573002D2D77616974002D740066696C746572002D4E00444F434B45522D49534F4C4154494F4E2D53544147452D31
Nov 09 13:11:45 CMX50070-101776 avahi-daemon[379]: New relevant interface docker0.IPv4 for mDNS.
Nov 09 13:11:45 CMX50070-101776

In [30]:
def answer_question(model, tokenizer, context, question):
    # Encode the context and question
    encoded = tokenizer.encode_plus(question, context, truncation=True, padding='max_length', max_length=512, return_tensors='pt', add_special_tokens = True)
    # Get the start and end scores for all tokens
    result = model(**encoded)
    start_scores = result["start_logits"]
    end_scores = result["end_logits"]

    # Find the tokens with the highest start and end scores
    answer_start = torch.argmax(start_scores)
    answer_end = torch.argmax(end_scores)

    # If the end score is higher than the start score, swap them
    if answer_end < answer_start:
        answer_start, answer_end = answer_end, answer_start

    # Get the tokens for the answer
    all_tokens = tokenizer.convert_ids_to_tokens(encoded['input_ids'][0])
    answer = ' '.join(all_tokens[answer_start : answer_end+1])

    return answer

def calculate_similarity(question_vector, answer_vector):
    similarity = 1 - cosine(question_vector[0], answer_vector[0])

    return similarity


def find_best_answer(model, tokenizer, context, question, model_vec, num_answers=3, overlap=50, max_length=512):

    # Vectorize the question
    question_vector = vectorize_text(model_vec, tokenizer, question)
    
    # Initialize the best answers and their similarities to the question
    best_answers = [(None, -1) for _ in range(num_answers)]
    
    # Split the context into chunks
    chunks = split_context(context, 1)
    
    for chunk in chunks:
        answer = answer_question(model, tokenizer, chunk, question)
        if answer is not None:
            answer_vector = vectorize_text(model_vec, tokenizer, answer)
            if answer_vector is not None:
                similarity = calculate_similarity(question_vector, answer_vector)
                # Check if the similarity is higher than the current lowest in best_answers
                if similarity > best_answers[0][1]:
                    # Replace the lowest
                    best_answers[0] = (answer.replace(" ##", ''), similarity)
                    # Sort the list so the lowest similarity is first
                    best_answers = sorted(best_answers, key=lambda x: x[1])
    # Return the answers along with their similarities
    return best_answers

def vectorize_text(model, tokenizer, input_string):
    # Encode the input string
    inputs = tokenizer.encode_plus(
        input_string,
        add_special_tokens=True,
        return_tensors="pt",
        padding=True, truncation=True,max_length=512
    )
    # Get the output from the model
    outputs = model(**inputs)
    # Get the embeddings from the last hidden state
    embeddings = outputs.last_hidden_state
    # Average the embeddings
    vector = torch.mean(embeddings, dim=1)
    # Convert tensor to numpy array
    vector = vector.detach().numpy()
    return vector

def split_context(context, chunk_size):
    lines = context.splitlines()
    result = []
    for i in range(0, len(lines), chunk_size):
        chunk = lines[i:i + chunk_size]
        merged_text = '\n'.join(chunk)
        result.append(merged_text)
    return result

In [28]:
from transformers import AutoTokenizer, AutoModelForQuestionAnswering, AutoModel

model_name = "bert-large-uncased-whole-word-masking-finetuned-squad"
model = AutoModelForQuestionAnswering.from_pretrained(model_name) #used for answering
model_vec = AutoModel.from_pretrained(model_name) #used for vectorization
tokenizer = AutoTokenizer.from_pretrained(model_name) #used for tokenization

Some weights of the model checkpoint at bert-large-uncased-whole-word-masking-finetuned-squad were not used when initializing BertForQuestionAnswering: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [31]:
question = "What is the error?"

best_answer = find_best_answer(model, tokenizer, 
                               summary, question, 
                               model_vec, 
                               num_answers=3, 
                               overlap=100, 
                               max_length=tokenizer.model_max_length)
print(f"The best answers are: {best_answer}")

The best answers are: [('state change : prepare - > config', 0.7152079343795776), ('startup done . ready to receive requests', 0.7248512506484985), ('to make sure that you have the latest version', 0.7656605839729309)]
